In [60]:
from models.TrackedObject import TrackedObject
from models.VideoAnalysis import VideoAnalysis
from utils.dynamics import EuclideanDistTracker, get_overlap_info
import utils.files as fil
import cv2
import os
import analysis.mask_rcnn as mask_rcnn
from analysis import yolo
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
isColab=False
if isColab:
    from google.colab.patches import cv2_imshow
    from google.colab import drive
from IPython.display import display, clear_output
import PIL.Image

tracker = EuclideanDistTracker()
transform, COCO_INSTANCE_CATEGORY_NAMES, model, device= mask_rcnn.config()
model= yolo.config()


if isColab:
    # Process video frames
    drive.mount('/content/drive/',force_remount=True)
    frames_folder = '/content/drive/MyDrive/Yolo/001102'
else:
    frames_folder = './data/accident/cadp_001102'



frame_filenames= sorted(
    [f for f in os.listdir(frames_folder) if f.endswith('.jpg')],
    key=fil.extract_number
)
def process_clip(frame_filenames):
    frame_filenames = [os.path.join(frames_folder, f) for f in frame_filenames]
    va=VideoAnalysis()
    # Get frame size dynamically from the first frame
    sample_frame = cv2.imread(frame_filenames[0])
    height, width = sample_frame.shape[:2]
    frame_size = (width, height)
    fps = 18

    # Save output to MP4 instead of AVI
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or use 'avc1' for better compatibility
    out = cv2.VideoWriter("output_file.mp4", fourcc, fps, frame_size)

    for frame_filename in frame_filenames:
        frame = cv2.imread(frame_filename)
        if frame is None:
            continue
        #result_frame,_,_objects = mask_rcnn.process_frame(model, frame, device, transform, COCO_INSTANCE_CATEGORY_NAMES, tracker, conf=0.5)
        result_frame, _objects = yolo.process_frame(model, frame, device, transform, COCO_INSTANCE_CATEGORY_NAMES, tracker, conf=0.5)
        prev_objects=_objects
        for o in va.allObjects:
          o.is_present=False
          o.is_accident=False
        for obj in _objects:
          if obj.id>(len(va.allObjects)-1):
            va.allObjects.append(obj)
          else:
            o=va.allObjects[obj.id]
            o.id=obj.id
            o.is_present=obj.is_present
            o.x1=obj.x1
            o.x2=obj.x2
            o.y1=obj.y1
            o.y2=obj.y2
            o.centroid=obj.centroid
            o.overlaps=obj.overlaps
            o.past_centroids.append(obj.centroid)
            o.compute_speed()
            o.compute_acceleration()
            o.check_accident()
            #print(str(o.id)+str(o.is_accident)+str(o.centroid))
        for box in va.allObjects:
            #cv2.putText(result_frame, str(box.acceleration),(box.x1, box.y1 - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), 2)
            if box.is_accident:
              cv2.rectangle(result_frame, (box.x1, box.y1), (box.x2, box.y2), (0, 255, 255), 3)
        #print(frame_filename)
        out.write(result_frame)
        if isColab:
            #Convert to PIL format for proper display in Jupyter/Colab
            #Display only the current frame
            #clear_output(wait=True)
            result_frame_rgb = cv2.cvtColor(result_frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            result_pil = PIL.Image.fromarray(result_frame_rgb)
            display(result_pil)
        else:
            cv2.imshow('res',result_frame)
            if cv2.waitKey(30) & 0xFF == ord('q'):
                  break
    out.release()
    cv2.destroyAllWindows()


Ultralytics 8.3.40  Python-3.12.7 torch-2.5.1+cpu CPU (12th Gen Intel Core(TM) i5-1235U)
Setup complete  (12 CPUs, 15.7 GB RAM, 397.8/454.3 GB disk)


In [62]:
process_clip(frame_filenames)


0: 384x640 (no detections), 379.9ms
Speed: 15.7ms preprocess, 379.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 391.1ms
Speed: 0.0ms preprocess, 391.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 349.7ms
Speed: 0.0ms preprocess, 349.7ms inference, 13.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 329.3ms
Speed: 0.0ms preprocess, 329.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 322.9ms
Speed: 2.0ms preprocess, 322.9ms inference, 6.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 340.6ms
Speed: 8.5ms preprocess, 340.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 332.3ms
Speed: 0.0ms preprocess, 332.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 349.1ms
Speed: 0.0ms pre